In [1]:
import time
import pandas as pd
from datetime import datetime
import joblib
import os.path

#Firebase Libraries
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db

#Machine Leearning Libraries
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn import model_selection
from sklearn import metrics
from sklearn import datasets

#Fetch the service account key JSON file contents
cred = credentials.Certificate("serviceAccountKey.json")

#initialize the app if not already
if not firebase_admin._apps:
    firebase_admin.initialize_app(cred, {
        'databaseURL': 'https://smartyblinds-default-rtdb.firebaseio.com/'
    })
    
SERIAL_NUM = 123

week_min = 0

#Run infinitely 
while(1):

    #Repeat every 10 minutes
    #time.sleep(10*60)
    time.sleep(1)
    
    week_min = week_min + 10
    
    print(week_min)

    #Store sensor data from firebase & Train with the dataframe 
    #only if we have at least 50 data points
    #and if the current week is different than the most recent data point week
    #(collect and train every week)

    ref_total_data = db.reference('Blinds/' + str(SERIAL_NUM) + '/Total_Data')
    total_data = ref_total_data.get()

    ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Data/' + str(total_data) + '/RTC')
    recent = ref_event.get()
    recent_week = datetime.strptime(str(recent), "%Y-%m-%d %H:%M:%S")

    today_week = datetime.today()
    #print("Today date is: ", recent_week.isocalendar().week)
    #print("Today date is: ", today_week.isocalendar().week)
    #print("Today date is: ", (today_week.isocalendar().week)!=(recent_week.isocalendar().week))

    if ((total_data >= 50) and ((today_week.isocalendar().week)!=(recent_week.isocalendar().week+2)) and (week_min == 100) ):
        print("s")

        dictionary = {"Event":[],"Light":[],"RTC":[],"Temp":[]}

        for i in range(total_data - 1):
            ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Data/' + str(i + 1) + '/Event')
            event = ref_event.get()
            dictionary['Event'].append(event)

            ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Data/' + str(i + 1) + '/Light')
            event = ref_event.get()
            dictionary['Light'].append(event)

            ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Data/' + str(i + 1) + '/RTC')
            event = ref_event.get()
            dateOBJ = datetime.strptime(str(event), "%Y-%m-%d %H:%M:%S")
            dictionary['RTC'].append(dateOBJ)

            ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Data/' + str(i + 1) + '/Temp')
            event = ref_event.get()
            dictionary['Temp'].append(event)

        #Saved sensor information into dataframe.
        df = pd.DataFrame(dictionary)

        #Convert into readable dataframe for ML algorithm
        df['Day'] = df['RTC'].dt.dayofweek
        df['Month'] = df['RTC'].dt.month
        df['Minutes'] = ((df['RTC'].dt.hour * 60) + df['RTC'].dt.minute)

        display(df)

        #Save data from firebase onto local server csv
        #df.to_csv('Data_' + str(SERIAL_NUM),index = False)

        #Retrieve saved data from local server csv
        #df = pd.read_csv('Data_' + str(SERIAL_NUM))
        
        
        #Training with the dataframe 
        
        #Initialize variables for the final decision tree parameters
        crit = 0
        max_split = 0
        max_rand = 0
        max_leaf = 0
        max_deep = 0
        max_accuracy = 0

        #Split dataset in features and target variable
        conditions = ['Light', 'Temp', 'Day', 'Minutes']
        X = df[conditions].values
        y = df.Event

        #Nested for loop used to cycle through multiple different parameters to determine the best results
        for i in range(2, 4):
            for j in range(0, 43):
                for k in range(0, 2):
                    for l in range(2, 200):
                        for m in range(1, 20):

                            #Splits the data into a training set and testing set
                            x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=(i/10), random_state=j, shuffle=True)

                            #If statement which checks if the criterion used for the tree should be 'gini' or 'entropy'
                            if k == 0:
                                testtree = DecisionTreeClassifier(criterion="gini", max_leaf_nodes=l, max_depth=m, splitter="best")
                            elif k == 1:
                                testtree = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=l, max_depth=m, splitter="best")

                            #Fits the data into a decision tree
                            testtree.fit(x_train, y_train)

                            #Prediction gives the accuracy of the decision tree based on the testing data
                            prediction = testtree.predict(x_test)
                            accuracy = metrics.accuracy_score(y_test, prediction)

                            #If statement which saves the values of the parameters which gives the highest accuracy into variables
                            if accuracy > max_accuracy:
                                max_accuracy = accuracy
                                max_split = i/10
                                max_rand = j
                                max_leaf = l
                                max_deep = m

                                if k == 0:
                                    crit = 0
                                elif k == 1:
                                    crit = 1

        #Prints out the highest accuracy possible for this data set
        print("Accuracy:", max_accuracy)

        #Splits the data in training and testing using the parameters which gave the highest accuracy
        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=max_split, random_state=max_rand, shuffle=True)

        #Creates a decision tree using the paramters which gave the highest accuracy
        if crit == 0:
            dtree = DecisionTreeClassifier(criterion="gini", max_leaf_nodes=max_leaf, max_depth=max_deep, splitter="best")
        elif crit == 1:
            dtree = DecisionTreeClassifier(criterion="entropy", max_leaf_nodes=max_leaf, max_depth=max_deep, splitter="best")

        #Creates the final model using the training data
        dtree.fit(x_train, y_train)

        #save the model
        joblib.dump(dtree, 'Tree_Model_'+ str(SERIAL_NUM) +'.sav')

        #Predict the response for test dataset
        y_pred = dtree.predict(x_test)
        #Model Accuracy, how often is the classifier correct?
        print("Accuracy:",str(round(metrics.accuracy_score(y_test, y_pred)*100)) + "%")
        #Visualizing Decision Trees
        #fig = plt.figure(figsize=(25,20), dpi=200)
        #tree.plot_tree(dtree, feature_names=conditions, filled=True)
        
        
        week_min = 0


    #Automated process, only if we have trained a model

    if(os.path.isfile('Tree_Model_'+ str(SERIAL_NUM) +'.sav')):

        #Retrieve Model
        dtree = joblib.load('Tree_Model_'+ str(SERIAL_NUM) +'.sav')

        #Get Current sensor data
        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Light')
        current_light = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/Temp')
        current_temperature = ref_event.get()

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Current/RTC')
        current_RTC = ref_event.get()
        dateOBJ = datetime.strptime(str(current_RTC), "%Y-%m-%d %H:%M:%S")

        day = dateOBJ.weekday()

        month = dateOBJ.month

        minutes = (dateOBJ.hour * 60) + (dateOBJ.minute)


        current_test = [[current_light,current_temperature,day,1000]]

        print(current_test)

        #Current sensor data is tested with the model
        new_output = dtree.predict(current_test)

        print(new_output)

        ref_event = db.reference('Blinds/' + str(SERIAL_NUM) + '/Blind_State')
        blind_event = ref_event.get()

        if (new_output.item(0) != blind_event):

            #control blinds automatically
            ref = db.reference('Blinds/' + str(SERIAL_NUM) + '/')
            ref.update({"Operation": "Auto"})
            ref.update({"Blind_State": new_output.item(0)})


    #https://github.com/firebase/firebase-admin-python
    #https://github.com/matplotlib/matplotlib
    #https://github.com/pandas-dev/pandas
    #https://github.com/scikit-learn/scikit-learn

10
[[479, 24.4, 1, 1000]]
[0.]
20
[[479, 24.4, 1, 1000]]
[0.]
30
[[479, 24.4, 1, 1000]]
[0.]
40
[[479, 24.4, 1, 1000]]
[0.]
50
[[479, 24.4, 1, 1000]]
[0.]
60
[[479, 24.4, 1, 1000]]
[0.]
70
[[479, 24.4, 1, 1000]]
[0.]
80
[[479, 24.4, 1, 1000]]
[0.]
90
[[479, 24.4, 1, 1000]]
[0.]
100
s


,Event,Light,RTC,Temp,Day,Month,Minutes
0,1.0,591.0,2022-12-01 06:00:00,23.1,3,12,360
1,0.0,626.0,2022-12-01 08:30:00,22.0,3,12,510
2,1.0,546.0,2022-12-01 17:00:00,21.5,3,12,1020
3,0.0,546.0,2022-12-01 17:30:00,23.1,3,12,1050
4,1.0,549.0,2022-12-01 21:50:00,21.3,3,12,1310
...,...,...,...,...,...,...,...
411,0.0,557.0,2023-02-26 18:50:00,21.1,6,2,1130
412,1.0,603.0,2023-02-26 19:10:00,22.1,6,2,1150
413,0.0,491.0,2023-02-26 19:40:00,23.4,6,2,1180
414,1.0,563.0,2023-02-27 06:40:00,20.0,0,2,400


Accuracy: 0.6428571428571429
Accuracy: 61%
[[479, 24.4, 1, 1000]]
[0.]
10
[[479, 24.4, 1, 1000]]
[0.]
20
[[479, 24.4, 1, 1000]]
[0.]
30
[[479, 24.4, 1, 1000]]
[0.]
40
[[479, 24.4, 1, 1000]]
[0.]
50
[[479, 24.4, 1, 1000]]
[0.]
60
[[479, 24.4, 1, 1000]]
[0.]
70
[[479, 24.4, 1, 1000]]
[0.]
80
[[479, 24.4, 1, 1000]]
[0.]
90
[[479, 24.4, 1, 1000]]
[0.]
100
s


KeyboardInterrupt: 